# Imports

In [1]:
%matplotlib qt
%load_ext autoreload   
%autoreload 2


In [2]:
from vpm_py import VPM
import numpy as np
from mpi4py import MPI

from vpm_py.console_io import print_IMPORTANT, print_red, print_blue, print_green
from vpm_py.visualization import StandardVisualizer
from test_problems.hill_vortex import hill_assign_parallel

# Parameters

In [3]:
UINF = np.array([0.0, 0.0, 1.0])
REYNOLDS_NUMBER = 100 #np.inf #0.1 
SPHERE_RADIUS = 2.0
# Reynolds number = U * L / nu , where U is the velocity, L is the radius of the sphere and nu is the kinematic viscosity
# nu = U * L / REYNOLDS_NUMBER
VISCOSITY = np.linalg.norm(UINF) * SPHERE_RADIUS / REYNOLDS_NUMBER
# DT should be set according to the CFL condition: CFL = U * DT / dx < 1
DT = 0.5 * 0.1 / np.linalg.norm(UINF)
MAX_ITER = 500
CFL_LIMITS = [0.3 , 0.9]
CFL_TARGET = 0.6

# OPTIONS
remesh = True
apply_vorticity_correction = False

# CASE FOLDER
CASE_FOLDER = "/mnt/c/Users/tryfonas/Data/hill_vortex"
if REYNOLDS_NUMBER == np.inf:
    CASE_FOLDER += "_Re=inf"
else:
    CASE_FOLDER += f"_Re={REYNOLDS_NUMBER}"

if apply_vorticity_correction:
    CASE_FOLDER += "_correct"
else:
    CASE_FOLDER += "_nocorrect"

if not remesh:
    CASE_FOLDER += "_no_remesh"

CASE_FOLDER += "/"

# Initialize MPI
comm = MPI.COMM_WORLD
start_time = MPI.Wtime()
rank = comm.Get_rank()
np_procs = comm.Get_size()


# Initialize VPM

In [ ]:

# Initialize VPM
vpm = VPM(
    number_of_equations=3,
    number_of_processors=np_procs,
    rank=rank,
    verbocity=0,
    dx_particle_mesh=0.1,
    dy_particle_mesh=0.1,
    dz_particle_mesh=0.1,
    case_folder=CASE_FOLDER,
)
plotter = StandardVisualizer(
    plot_particles=("charge", "magnitude"),  # plot_mesh=("velocity", "magnitude")
)
vpm.attach_visualizer(plotter)


# PRINT THE RANK OF THE PROCESS AND DETERMINE HOW MANY PROCESSES ARE RUNNING
print_blue(f"Number of processes: {np_procs}", rank)
comm.Barrier()
print_blue(f"Rank: {rank}")
comm.Barrier()


In [ ]:
DT = 1 * 0.1
NI = -0.1
neq = 3
UINF = np.array([0.0, 0.0, 0.0])

# Create particles
NVR = 100
XPR_zero = np.zeros((3, NVR), dtype=np.float64)
XPR_zero[:, 0] = np.array([-2, -2, -20])
XPR_zero[:, 1] = np.array([2, 2, 20])
QPR_zero = np.ones((neq + 1, NVR), dtype=np.float64)
UPR_zero = np.zeros((3, NVR), dtype=np.float64)
GPR_zero = np.zeros((3, NVR), dtype=np.float64)


In [ ]:
# Initialization VPM
comm.Barrier()
vpm.vpm_define(
    num_equations=neq,
    particle_positions=XPR_zero,
    particle_charges=QPR_zero,
)
comm.Barrier()


# Initialize Hill Vortex

In [ ]:
if rank == 0:
    st = MPI.Wtime()

print_IMPORTANT("Hill vortex initialization", rank)
_, RHS_pm_1 = hill_assign_parallel(
    Dpm=vpm.dpm,
    NN=vpm.particle_mesh.nn,
    NN_bl=vpm.particle_mesh.nn_bl,
    Xbound=vpm.particle_mesh.xbound,
    neqpm=vpm.num_equations,
    sphere_radius=1.0,
    u_freestream=1.0,
    sphere_z_center=0.0,
)
RHS_pm_hill = RHS_pm_1
# visualize_vorticity(RHS_pm_hill, vpm.nn_bl)


In [ ]:
vpm.particle_mesh.set_rhs_pm(RHS_pm_hill)
print_red("Setting RHS_PM as computed from the hill vortex", rank)

if rank == 0:
    st = MPI.Wtime()
    print_red("Remeshing")
XPR_zero, QPR_zero = vpm.remesh_particles(project_particles=False)
if rank == 0:
    et = MPI.Wtime()
    print(f"\tRemeshing finished in {int((et - st) / 60)}m {int(et - st) % 60}s\n")

print_IMPORTANT("Particles initialized", rank)

# Create the plot to live update the particles
if rank == 0:
    remesh_str = 'remesh = True' if remesh else 'remesh = False'
    correct_str = 'correction = True' if apply_vorticity_correction else 'correction = False'
   
    vpm.update_plot(
        f"Reynolds {REYNOLDS_NUMBER} |  Time: {0:.2f}s | Iteration: {0}/{MAX_ITER} | {remesh_str} | {correct_str}",
    ) 


In [9]:
vpm.vpm_define(
    num_equations= vpm.num_equations,
    particle_positions  =  XPR_zero[:,:],
    particle_charges    =  QPR_zero[:,:],
)
# Main loop
T = 0
XPR = XPR_zero.copy()
QPR = QPR_zero.copy()
i = 0


# Simulate

In [ ]:
i += 1
NVR = vpm.particles.NVR
XPR = vpm.particles.XP
QPR = vpm.particles.QP

T += DT
print_IMPORTANT(
    f"Iteration= {i} of {MAX_ITER}\nT={T}\nDT={DT}",
    rank = rank,
    color_divider="green",
    color_text="green"
)

vpm.vpm(
    num_equations=neq,
    mode = 2,
    particle_positions    =  XPR,
    particle_charges      =  QPR,
    timestep=i,
    viscosity=NI,
)

In [ ]:
vpm.particles.particle_positions.shape

In [ ]:
print_IMPORTANT("INFO", rank)
XPR = vpm.particles.XP
QPR = vpm.particles.QP
UPR = vpm.particles.UP.to_numpy(copy=True)
GPR = vpm.particles.GP.to_numpy(copy=True)
# Print the size of the particles
# print(f"Number of particles: {NVR}")
# print(f"Number of equations: {neq}")
# print('\n')

# print_green(f"UPR:")
# print(f"Mean: {np.mean(UPR.data, axis=1)}")
# print(f"Max: {np.max(UPR.data, axis=1)}")
# print(f"Min: {np.min(UPR.data, axis=1)}")
# print('\n')

U_PM = vpm.particle_mesh.U
for name, u in zip(["Ux", "Uy", "Uz"], U_PM): 
    print_green(f"{name}:")
    print(f"Mean: {np.mean(u)}")
    print(f"Max: {np.max(u)}")
    print(f"Min: {np.min(u)}")
    print('\n')

print_IMPORTANT("Convecting Particles", rank)

st = MPI.Wtime()
# # Move the particles
for j in range(vpm.particles.NVR):
    # Translate the particles
    XPR[:3, j] = XPR[:3, j] + UPR[:3,j] * DT
    pass
    # QPR[:3, j] -= GPR[:3, j] * DT
et = MPI.Wtime()

print(f"\tConvection finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")
print_IMPORTANT("Updating the plot", rank)

st = MPI.Wtime()
# Update the plot
vpm.update_plot(
    f"Reynolds {REYNOLDS_NUMBER} |  Time: {T:.2f}s | Iteration: {i}/{MAX_ITER} | {remesh_str} | {correct_str}",
)
et = MPI.Wtime()
print(f"\tUpdating the plot finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")
print_IMPORTANT("Saving the particles and particle mesh", rank)

st = MPI.Wtime()
vpm.particles.save_to_file(filename= "particles_test", folder="vpm_case")
vpm.particle_mesh.save_to_file(filename= "particle_mesh_test", folder="vpm_case")
et = MPI.Wtime()
print(f"\tSaving the particles and particle mesh finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")


## Remesh and Redefine

In [12]:
vpm.vpm_define(
    num_equations=neq,
    particle_positions  =  XPR,
    particle_charges    =  QPR,
    timestep=i,
)
# XPR, QPR = vpm.remesh_particles(project_particles=True, cut_off=1e-5)
